In [1]:
# Install dependencies
!pip install ninja yacs cython matplotlib demjson
!pip install git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI

     |████████████████████████████████| 102kB 4.6MB/s ta 0:00:011
     |████████████████████████████████| 2.1MB 15.7MB/s eta 0:00:01
     |████████████████████████████████| 13.1MB 55.1MB/s eta 0:00:01
     |████████████████████████████████| 92kB 37.4MB/s eta 0:00:01
     |████████████████████████████████| 71kB 1.6MB/s  eta 0:00:01
  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-ia_eas9b
  Running command git clone -q https://github.com/cocodataset/cocoapi.git /tmp/pip-req-build-ia_eas9b
  Stored in directory: /tmp/pip-ephem-wheel-cache-5njxgh30/wheels/90/51/41/646daf401c3bc408ff10de34ec76587a9b3ebfac8d21ca5c3a
Successfully built pycocotools


In [2]:
!ls

build		    docs	     README.md	       tools
CODE_OF_CONDUCT.md  fastText	     requirements.txt  Untitled.ipynb
configs		    LICENSE	     save	       vqa-maskrcnn-benchmark
CONTRIBUTING.md     pythia	     setup.py
data		    pythia.egg-info  tests


In [3]:
%cd vqa-maskrcnn-benchmark

/home/rmandli_g_clemson_edu/pythia/vqa-maskrcnn-benchmark


In [4]:
!ls

ABSTRACTIONS.md     demo			 MODEL_ZOO.md
build		    docker			 README.md
CODE_OF_CONDUCT.md  INSTALL.md			 setup.py
configs		    LICENSE			 tests
CONTRIBUTING.md     maskrcnn_benchmark		 tools
create_json.py	    maskrcnn_benchmark.egg-info  TROUBLESHOOTING.md


In [7]:
!sudo python setup.py build
!sudo python setup.py develop

running build
running build_py
running build_ext
running develop
running egg_info
writing maskrcnn_benchmark.egg-info/PKG-INFO
writing top-level names to maskrcnn_benchmark.egg-info/top_level.txt
writing dependency_links to maskrcnn_benchmark.egg-info/dependency_links.txt
reading manifest file 'maskrcnn_benchmark.egg-info/SOURCES.txt'
writing manifest file 'maskrcnn_benchmark.egg-info/SOURCES.txt'
running build_ext
copying build/lib.linux-x86_64-2.7/maskrcnn_benchmark/_C.so -> maskrcnn_benchmark
Creating /usr/local/lib/python2.7/dist-packages/maskrcnn-benchmark.egg-link (link to .)
maskrcnn-benchmark 0.1 is already the active version in easy-install.pth

Installed /home/rmandli_g_clemson_edu/pythia/vqa-maskrcnn-benchmark
Processing dependencies for maskrcnn-benchmark==0.1
Finished processing dependencies for maskrcnn-benchmark==0.1


In [10]:
import sys
sys.path.append('/home/rmandli_g_clemson_edu/pythia')
sys.path.append('/home/rmandli_g_clemson_edu/pythia/vqa-maskrcnn-benchmark')

In [87]:
import yaml
#import cv2
import torch
import requests
import numpy as np
import gc
import torch.nn.functional as F
#import pandas as pd

import torchvision.models as models
import torchvision.transforms as transforms


from PIL import Image
from IPython.display import display, HTML, clear_output
from ipywidgets import widgets, Layout
from io import BytesIO

# from maskrcnn_benchmark.config import cfg
# from maskrcnn_benchmark.layers import nms
# from maskrcnn_benchmark.modeling.detector import build_detection_model
# from maskrcnn_benchmark.structures.image_list import to_image_list
# from maskrcnn_benchmark.utils.model_serialization import load_state_dict

from pythia.utils.configuration import ConfigNode
from pythia.tasks.processors import VocabProcessor, VQAAnswerProcessor, FastTextProcessor
from pythia.models import LoRRA
from pythia.common.registry import registry
from pythia.common.sample import Sample, SampleList
from pprint import pprint

In [88]:
!pwd
!ls configs/vqa/textvqa/lorra.yml

/home/rmandli_g_clemson_edu/pythia
configs/vqa/textvqa/lorra.yml


In [89]:
pythia_path = '/home/rmandli_g_clemson_edu/pythia'
model_config_path = '/home/rmandli_g_clemson_edu/pythia/configs/vqa/textvqa/lorra.yml'

In [66]:
!wget -O data/vocabulary_100k.txt https://dl.fbaipublicfiles.com/pythia/data/vocabulary_100k.txt

--2019-06-02 10:40:53--  https://dl.fbaipublicfiles.com/pythia/data/vocabulary_100k.txt
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.20.22.166, 104.20.6.166, 2606:4700:10::6814:16a6, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.20.22.166|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 626738 (612K) [text/plain]
Saving to: ‘data/vocabulary_100k.txt’

data/vocabulary_100 100%[===================>] 612.05K  --.-KB/s    in 0.03s   

2019-06-02 10:40:53 (18.6 MB/s) - ‘data/vocabulary_100k.txt’ saved [626738/626738]



In [122]:
class TextVQADemo:
    def __init__(self):
        self._init_processors()
        self.lorra_model = self._build_lorra_model()
#         self.detection_model = self._build_detection_model()
#         self.resnet_model = self._build_resnet_model()
        
    def _init_processors(self):
        """
        Pythia uses processors is to keep data processing pipelines as similar as
        possible for different datasets and allow code reusability.
        """
        with open("configs/vqa/textvqa/lorra.yml") as f:
            config = yaml.load(f, Loader=yaml.FullLoader)
            
        config['datasets'] = 'textvqa'
        #update config with includes for model specific config
        for inc in config.get("includes", []):
            config.update(yaml.load(open("pythia/"+ inc), Loader=yaml.FullLoader))

        config = ConfigNode(config)
        textvqa_config = config.task_attributes.vqa.dataset_attributes.textvqa
        
        answer_processor_config = textvqa_config.processors.answer_processor
        answer_processor_config.params.vocab_file = \
            "data/vocabs/answers_textvqa_more_than_1.txt"
        self.answer_processor = VQAAnswerProcessor(answer_processor_config.params)
        registry.register("textvqa_answer_processor", self.answer_processor)
        
        text_processor_config = textvqa_config.processors.text_processor
        text_processor_config.params.vocab.vocab_file = "data/vocabs/vocabulary_100k.txt"
        self.text_processor = VocabProcessor(text_processor_config.params)
        registry.register("textvqa_text_processor", self.text_processor)
        pprint(text_processor_config)
        
        context_processor_config = textvqa_config.processors.context_processor
        self.context_processor = FastTextProcessor(context_processor_config.params)
        registry.register("textvqa_context_processor", self.context_processor)
        
        self.config = config

        registry.register("config", config)
        #pprint(config.classifier.keys())
        
    def _build_lorra_model(self):
        state_dict = torch.load('data/models/lorra_best.pth')
        model_config = self.config.model_attributes.lorra
        pprint(model_config.classifier)
        model = LoRRA(model_config)
        model.build()
        model.init_losses_and_metrics()
        if list(state_dict.keys())[0].startswith('module') and \
           not hasattr(model, 'module'):
            state_dict = self._multi_gpu_state_to_single(state_dict)

        model.load_state_dict(state_dict)
        model.to("cuda")
        model.eval()
    
demo = TextVQADemo()

{'params': {'max_length': 14,
            'preprocessor': {'params': ConfigNode(ConfigNode()),
                             'type': 'simple_sentence'},
            'vocab': {'embedding_name': 'glove.6B.300d',
                      'type': 'intersected',
                      'vocab_file': 'data/vocabs/vocabulary_100k.txt'}},
 'type': 'vocab'}


RuntimeError: CUDA out of memory. Tried to allocate 86.50 MiB (GPU 0; 11.17 GiB total capacity; 10.82 GiB already allocated; 21.31 MiB free; 206.00 KiB cached)